Package Load

In [2]:
import torch
print('pytorch version: {}'.format(torch.__version__))

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
%matplotlib inline

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

# 이걸 해줘야 matplotlib 시행 시 에러가 안 남
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

pytorch version: 2.1.2
pytorch version: 2.1.2
GPU 사용 가능 여부: True


Hyperparameters

In [3]:
batch_size = 64
num_epochs = 50
learning_rate = 0.001

Dataloader

In [4]:
# For Kaggle notebook
data_dir = '/kaggle/input/kermany2018/OCT2017 /'

In [5]:
class OCTDataset(Dataset):
    def __init__(self, data_dir, mode, transform=None):
        self.all_data = sorted(glob.glob(os.path.join(data_dir, mode,'*', '*')))
        self.transform = transform
    
    def __getitem__(self, index):
        data_path = self.all_data[index]
        img = Image.open(data_path)
        if self.transform is not None:
            img = self.transform(img)
        img = img.squeeze()
        img = torch.stack((img, img, img,), dim=0)  # VGG16이 3 channel이라 concat시키기
        name = os.path.basename(data_path)
        if name.startswith('NORMAL'):
            label = 0
        elif name.startswith('CNV'):
            label = 1
        elif name.startswith('DME'):
            label = 2
        elif name.startswith('DRUSEN'):
            label = 3
        return img, label
    
    def __len__(self):
        length = len(self.all_data)
        return length

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomResizedCrop(299, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(0.1881,0.1850)
    ]),
    'val': transforms.Compose([
        transforms.Resize([350]),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(0.1881,0.1850)
    ])
}
# Inception v3는 (3,299,299)를 사용함

train_data = OCTDataset(data_dir=data_dir, mode='train', transform=data_transforms['train'])
val_data = OCTDataset(data_dir=data_dir, mode='val', transform=data_transforms['val'])
test_data = OCTDataset(data_dir=data_dir, mode='test', transform=data_transforms['val'])

# For Kaggle: 2 core CPU available
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=2)  # num_worker로 Multi Process Data Loading 구현
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2) # Kaggle CPU = 2 core
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)

train_mean = 0.1881
train_std = 0.1850

Model: Inception V3 Transfer Learning

In [7]:
from collections import namedtuple
InceptionOutputs = namedtuple("InceptionOutputs", ["logits", "aux_logits"])

class TransferInceptionV3(nn.Module):
    def __init__ (self, num_classes):
        super(TransferInceptionV3, self).__init__()
        self.inception = torchvision.models.inception_v3(pretrained=True)
        aux_in_feature = self.inception.AuxLogits.fc.in_features
        self.inception.AuxLogits.fc = nn.Linear(aux_in_feature, num_classes)
        final_in_feature = self.inception.fc.in_features
        self.inception.fc = nn.Linear(final_in_feature, num_classes)
        
    def forward(self,x):
        return self.inception(x)

inception_transfer = TransferInceptionV3(num_classes=4).to(device)

In [8]:
inception_transfer

TransferInceptionV3(
  (inception): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a

In [9]:
# Auxiliary logit + 이전 6e inception module (181 ~ 218) / Fc layer + 이전 7b, 7c inception module (237~292)
count = 0
for param in inception_transfer.inception.parameters():
    count += 1
    if (181 <= count and count <= 218):
        param.requires_grad = True
    elif (237 <= count and count <= 292):
        param.requires_grad = True
    else:
        param.requires_grad = False

for name, param in inception_transfer.inception.named_parameters():
    print(name, param.requires_grad)

Conv2d_1a_3x3.conv.weight False
Conv2d_1a_3x3.bn.weight False
Conv2d_1a_3x3.bn.bias False
Conv2d_2a_3x3.conv.weight False
Conv2d_2a_3x3.bn.weight False
Conv2d_2a_3x3.bn.bias False
Conv2d_2b_3x3.conv.weight False
Conv2d_2b_3x3.bn.weight False
Conv2d_2b_3x3.bn.bias False
Conv2d_3b_1x1.conv.weight False
Conv2d_3b_1x1.bn.weight False
Conv2d_3b_1x1.bn.bias False
Conv2d_4a_3x3.conv.weight False
Conv2d_4a_3x3.bn.weight False
Conv2d_4a_3x3.bn.bias False
Mixed_5b.branch1x1.conv.weight False
Mixed_5b.branch1x1.bn.weight False
Mixed_5b.branch1x1.bn.bias False
Mixed_5b.branch5x5_1.conv.weight False
Mixed_5b.branch5x5_1.bn.weight False
Mixed_5b.branch5x5_1.bn.bias False
Mixed_5b.branch5x5_2.conv.weight False
Mixed_5b.branch5x5_2.bn.weight False
Mixed_5b.branch5x5_2.bn.bias False
Mixed_5b.branch3x3dbl_1.conv.weight False
Mixed_5b.branch3x3dbl_1.bn.weight False
Mixed_5b.branch3x3dbl_1.bn.bias False
Mixed_5b.branch3x3dbl_2.conv.weight False
Mixed_5b.branch3x3dbl_2.bn.weight False
Mixed_5b.branch3x3dbl

In [10]:
# 7b inception module의 Conv layer를 다시 initialization
name_list=[]
for name, param in inception_transfer.inception.Mixed_7b.named_parameters():
    name_list = name.split('.')
    if name_list[1]=='conv':
        if name_list[2]=='weight':
            print(name)
            nn.init.xavier_uniform_(param)
            print(name+'의 conv filter initialization setting 완료')
            print()

# 7c inception module의 Conv layer를 다시 initialization
name_list=[]
for name, param in inception_transfer.inception.Mixed_7c.named_parameters():
    name_list = name.split('.')
    if name_list[1]=='conv':
        if name_list[2]=='weight':
            print(name)
            nn.init.xavier_uniform_(param)
            print(name+'의 conv filter initialization setting 완료')
            print()

branch1x1.conv.weight
branch1x1.conv.weight의 conv filter initialization setting 완료

branch3x3_1.conv.weight
branch3x3_1.conv.weight의 conv filter initialization setting 완료

branch3x3_2a.conv.weight
branch3x3_2a.conv.weight의 conv filter initialization setting 완료

branch3x3_2b.conv.weight
branch3x3_2b.conv.weight의 conv filter initialization setting 완료

branch3x3dbl_1.conv.weight
branch3x3dbl_1.conv.weight의 conv filter initialization setting 완료

branch3x3dbl_2.conv.weight
branch3x3dbl_2.conv.weight의 conv filter initialization setting 완료

branch3x3dbl_3a.conv.weight
branch3x3dbl_3a.conv.weight의 conv filter initialization setting 완료

branch3x3dbl_3b.conv.weight
branch3x3dbl_3b.conv.weight의 conv filter initialization setting 완료

branch_pool.conv.weight
branch_pool.conv.weight의 conv filter initialization setting 완료

branch1x1.conv.weight
branch1x1.conv.weight의 conv filter initialization setting 완료

branch3x3_1.conv.weight
branch3x3_1.conv.weight의 conv filter initialization setting 완료

branch3x

In [11]:
# 7b, 7c inception module의 batch normalization gamma, beta initialization
# Residual block이 없으니 모두 1로 initialization, beta는 0으로 initialization
name_list=[]
for name, param in inception_transfer.inception.Mixed_7b.named_parameters():
    name_list = name.split('.')
    if name_list[1]=='bn':
        if name_list[2]=='weight':
            print(name)
            nn.init.ones_(param)
            print(name+'의 gamma one setting 완료')
            print()
        elif name_list[2]=='bias':
            print(name)
            nn.init.zeros_(param)
            print(name+'의 beta zero setting 완료')
            print()

name_list=[]
for name, param in inception_transfer.inception.Mixed_7c.named_parameters():
    name_list = name.split('.')
    if name_list[1]=='bn':
        if name_list[2]=='weight':
            print(name)
            nn.init.ones_(param)
            print(name+'의 gamma one setting 완료')
            print()
        elif name_list[2]=='bias':
            print(name)
            nn.init.zeros_(param)
            print(name+'의 beta zero setting 완료')
            print()

branch1x1.bn.weight
branch1x1.bn.weight의 gamma one setting 완료

branch1x1.bn.bias
branch1x1.bn.bias의 beta zero setting 완료

branch3x3_1.bn.weight
branch3x3_1.bn.weight의 gamma one setting 완료

branch3x3_1.bn.bias
branch3x3_1.bn.bias의 beta zero setting 완료

branch3x3_2a.bn.weight
branch3x3_2a.bn.weight의 gamma one setting 완료

branch3x3_2a.bn.bias
branch3x3_2a.bn.bias의 beta zero setting 완료

branch3x3_2b.bn.weight
branch3x3_2b.bn.weight의 gamma one setting 완료

branch3x3_2b.bn.bias
branch3x3_2b.bn.bias의 beta zero setting 완료

branch3x3dbl_1.bn.weight
branch3x3dbl_1.bn.weight의 gamma one setting 완료

branch3x3dbl_1.bn.bias
branch3x3dbl_1.bn.bias의 beta zero setting 완료

branch3x3dbl_2.bn.weight
branch3x3dbl_2.bn.weight의 gamma one setting 완료

branch3x3dbl_2.bn.bias
branch3x3dbl_2.bn.bias의 beta zero setting 완료

branch3x3dbl_3a.bn.weight
branch3x3dbl_3a.bn.weight의 gamma one setting 완료

branch3x3dbl_3a.bn.bias
branch3x3dbl_3a.bn.bias의 beta zero setting 완료

branch3x3dbl_3b.bn.weight
branch3x3dbl_3b.bn.weigh

Checkpoint

In [12]:
def save_model(model, epoch, optimizer, epoch_loss, val_accuracy, saved_dir):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss
    }
    val_accuracy = round(val_accuracy,1)
    file_name = str(f"ckpoint_{epoch+1}_{val_accuracy}%.pt")
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point,output_path)

Tensorboard

In [13]:
logs_base_dir = "./logs/logs_transfer"
os.makedirs(logs_base_dir, exist_ok=True)

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(logs_base_dir)

2024-05-08 11:10:42.911582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 11:10:42.911709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 11:10:43.044939: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Train, Validation, Test

In [14]:
def train(num_epochs, model, data_loader, criterion, optimizer, scheduler, val_every, device, saved_dir, writer):
    print('Start training..')
    torch.cuda.empty_cache()
    best_loss = 9999999
    train_loss_list = []
    val_loss_list = []
    train_accuracy_list = []
    val_accuracy_list = []
    for epoch in tqdm(range(num_epochs), desc='epoch'):
        count = 0
        best_epoch = 0
        # running_loss = 0.0
        for i, (imgs, labels) in tqdm(enumerate(data_loader), desc="in epoch"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs, aux = model(imgs)       # 여기서 inception net의 특징이 나옴
            output_loss = criterion(outputs, labels)
            aux_loss = criterion(aux, labels)
            loss = output_loss + 0.3*aux_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Saving train loss
            train_loss_list.append(loss.item())

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax).float().mean()
            # Saving train accuracy
            train_accuracy_list.append(accuracy)

            if (i+1) % 30 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100))
            if (i+1) % 50 == 0:
                writer.add_scalar('Loss/Train', loss, i)
                writer.add_scalar('Accuracy/Train', accuracy, i)
            #if i % 30 == 29:
                #loss_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
                #running_loss = 0.0
        if (epoch + 1) % val_every == 0:
            avrg_loss, val_accuracy = validation(epoch + 1, model, val_loader, criterion, device)
            # Saving validation loss and accuracy
            val_loss_list.append(avrg_loss)
            writer.add_scalar('Loss/Validation', avrg_loss, epoch+1)
            val_accuracy_list.append(val_accuracy)
            writer.add_scalar('Accuracy/Validation', val_accuracy, epoch+1)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, epoch, optimizer, avrg_loss, val_accuracy, saved_dir)
                count = 0
                best_epoch = epoch + 1
            else:
                count += 1
                print(f'Best epoch does not appear. Previous best epoch:{best_epoch}, A number of epochs remain until early stop: {10-count}')
                if count >= 10:
                    print('Best performance does not occur within 10 epochs. Early stopping!!')
                    scheduler.step()
                    writer.flush()
                    return train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list
            ''' Only for local setting
            loss_tracker(loss_plt, torch.Tensor([avrg_loss]), torch.Tensor([epoch]))'''
    scheduler.step()
    writer.flush()
    return train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list

def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            
            outputs = model(imgs)       # eval을 했기 때문에 aux가 필요 없음
            loss = criterion(outputs, labels)
            
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        val_accuracy = correct / total * 100
        print('Validation #{}  Accuracy: {:.2f}%  Average Loss: {:.4f}'.format(epoch, val_accuracy, avrg_loss))
    model.train()
    return avrg_loss, val_accuracy

def test(model, data_loader, device):
    print('Start test..')
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        answer_list = []
        pred_list = []
        pred_proba_list = []
        
        for i, (imgs, labels) in tqdm(enumerate(data_loader)):
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)       # 여기도 eval을 하기 때문에 aux가 필요 없음

            pred_probability = F.softmax(outputs, dim=1)
            pred_proba_list.append(pred_probability.tolist())
            
            _, argmax = torch.max(outputs, 1)
            pred_list.append(argmax.tolist())

            total += imgs.size(0)
            correct += (labels == argmax).sum().item()
            
            answer_list.append(labels.tolist())
        
        print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))
    model.train()        
    return pred_list, answer_list, pred_proba_list

Loss, optimizer, directory for saving

In [15]:
torch.manual_seed(7777)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(inception_transfer.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0.00001)

val_every = 1
saved_dir = './saved'

Train

In [16]:
train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list = train(num_epochs, inception_transfer, train_loader, criterion, optimizer, scheduler, val_every, device, saved_dir, writer)

Start training..


epoch:   0%|          | 0/50 [00:00<?, ?it/s]

in epoch: 0it [00:00, ?it/s]

Epoch [1/50], Step [30/1305], Loss: 0.3552, Accuracy: 90.62%
Epoch [1/50], Step [60/1305], Loss: 0.2752, Accuracy: 95.31%
Epoch [1/50], Step [90/1305], Loss: 0.5236, Accuracy: 85.94%
Epoch [1/50], Step [120/1305], Loss: 0.3976, Accuracy: 89.06%
Epoch [1/50], Step [150/1305], Loss: 0.5068, Accuracy: 90.62%
Epoch [1/50], Step [180/1305], Loss: 0.3227, Accuracy: 89.06%
Epoch [1/50], Step [210/1305], Loss: 0.3037, Accuracy: 89.06%
Epoch [1/50], Step [240/1305], Loss: 0.2001, Accuracy: 95.31%
Epoch [1/50], Step [270/1305], Loss: 0.2407, Accuracy: 96.88%
Epoch [1/50], Step [300/1305], Loss: 0.2449, Accuracy: 96.88%
Epoch [1/50], Step [330/1305], Loss: 0.3068, Accuracy: 93.75%
Epoch [1/50], Step [360/1305], Loss: 0.2016, Accuracy: 93.75%
Epoch [1/50], Step [390/1305], Loss: 0.1526, Accuracy: 96.88%
Epoch [1/50], Step [420/1305], Loss: 0.0705, Accuracy: 98.44%
Epoch [1/50], Step [450/1305], Loss: 0.2164, Accuracy: 95.31%
Epoch [1/50], Step [480/1305], Loss: 0.2903, Accuracy: 96.88%
Epoch [1/50

in epoch: 0it [00:00, ?it/s]

...

in epoch: 0it [00:00, ?it/s]

Epoch [30/50], Step [30/1305], Loss: 0.0188, Accuracy: 100.00%
Epoch [30/50], Step [60/1305], Loss: 0.0085, Accuracy: 100.00%
Epoch [30/50], Step [90/1305], Loss: 0.0959, Accuracy: 98.44%
Epoch [30/50], Step [120/1305], Loss: 0.0229, Accuracy: 100.00%
Epoch [30/50], Step [150/1305], Loss: 0.0134, Accuracy: 100.00%
Epoch [30/50], Step [180/1305], Loss: 0.0208, Accuracy: 98.44%
Epoch [30/50], Step [210/1305], Loss: 0.1396, Accuracy: 98.44%
Epoch [30/50], Step [240/1305], Loss: 0.0010, Accuracy: 100.00%
Epoch [30/50], Step [270/1305], Loss: 0.0166, Accuracy: 100.00%
Epoch [30/50], Step [300/1305], Loss: 0.0006, Accuracy: 100.00%
Epoch [30/50], Step [330/1305], Loss: 0.0012, Accuracy: 100.00%
Epoch [30/50], Step [360/1305], Loss: 0.0379, Accuracy: 100.00%
Epoch [30/50], Step [390/1305], Loss: 0.0044, Accuracy: 100.00%
Epoch [30/50], Step [420/1305], Loss: 0.0436, Accuracy: 96.88%
Epoch [30/50], Step [450/1305], Loss: 0.0204, Accuracy: 100.00%
Epoch [30/50], Step [480/1305], Loss: 0.0094, A

in epoch: 0it [00:00, ?it/s]

Epoch [31/50], Step [30/1305], Loss: 0.1090, Accuracy: 96.88%


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10,5))
X1 = np.linspace(0, num_epochs, 500)
plt.subplot(1,2,1)
plt.ylabel('Loss')
plt.plot(X1, train_loss_list, label = 'Train')
plt.plot(X1, val_loss_list, label = 'Validation')
plt.subplot(1,2,2)
plt.ylabel('Accuracy')
X2 = np.linspace(1, num_epochs, num_epochs)
plt.plot(X2, train_accuracy_list, label = 'Train')
plt.plot(X2, val_accuracy_list, label = 'Validation')
# plt.savefig('graph.png',facecolor = 'w')
plt.show()

Zip all outcomes

In [ ]:
import shutil
shutil.make_archive('OUTPUT', 'zip', '/kaggle/working/')

In [ ]:
from IPython.display import FileLink
FileLink(r'OUTPUT.zip') # 여기에 zip파일 이름 넣기